In [ ]:
# 영어 단어를 핚국어 단어로 번역하는 프로그램

import tensorflow as tf
import numpy as np

# 영어 알파벳과 한글들을 나열한 뒤 한 글자씩 배열에 집어 넣는다.
# 그런 다음 배열에 넣은 글자들을 딕셔너리 형태로 변경 합니다.
# Sequence to Sequence 모델에는 특수한 심볼이 몇개 필요하다
# S: 디코딩 입력의 시작을 나타내는 심볼
# E: 디코딩 출력을 끝을 나타내는 심볼
# P: 현재 배치 데이터의 time step 크기보다 작은 경우 빈 시퀀스를 채우는 심볼
# 예) 현재 배치 데이터의 최대 크기가 4 인 경우
# word -> ['w', 'o', 'r', 'd']
# to -> ['t', 'o', 'P', 'P']
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)
print(num_dic)

# 영어를 한글로 번역하기 위한 학습 데이터
seq_data = [['word', '단어'], ['wood', '나무'],
           ['game', '놀이'], ['girl', '소녀'],
           ['kiss', '키스'], ['love', '사랑']]

# 입력 단어와 출력 단어를 한 글자씩 떼어낸 뒤 배열로 만든후에
# one-hot 인코딩 형식으로 만들어 주는 함수
# 데이터는 총 3개로 구성된다(인코더의 입력값, 디코더의 입력값과 출력값)
def make_batch(seq_data):
  input_batch = [] # 인코더 셀의 입력값
  output_batch = [] # 디코더 셀의 입력값
  target_batch = [] # 디코더 셀의 출력값
  
  for seq in seq_data:

    #1.인코더 셀의 입력값. 입력단어의 글자들을 한글자씩 떼어 배열로 만든다.
    input = [num_dic[n] for n in seq[0]]

    #2.디코더 셀의 입력값. 시작을 나타내는 S 심볼을 맨 앞에 붙여준다.
    output = [num_dic[n] for n in ('S' + seq[1])]

    #3.학습을 위해 비교할 디코더 셀의 출력값. 끝나는 것을 알려주기 위해 마지막에 E 를 붙인다.
    target = [num_dic[n] for n in (seq[1] + 'E')]

    #위에서 맊들어진 데이터를 one-hot 인코딩합니다.
    input_batch.append(np.eye(dic_len)[input])
    output_batch.append(np.eye(dic_len)[output])

    # 출력값맊 one-hot 인코딩이 아닌 인덱스 번호를 리턴 (sparse_softmax_cross_entropy_with_logits 사용)
    target_batch.append(target)

  return input_batch, output_batch, target_batch

#############
# 옵션 설정
#############
learning_rate = 0.01 # 학습률
n_hidden = 128 # 은닉층의 노드 갯수
total_epoch = 100 # 학습 횟수

# 입력과 출력의 형태가 one-hot 인코딩으로 같으므로 크기도 같다.
n_class = n_input = dic_len

####################
# 신경망 모델 구성
####################
# Seq2Seq 모델은 인코더의 입력과 디코더의 입력의 형식이 같다.
# 입력값들은 one-hot 인코딩을 사용하고, 디코더의 출력값은 인덱스 숫자를
# 그대로 사용하기 때문에 입력값의 랭크(차원)가 하나 더 높다.
# [batch size, time steps, input size] : 인코더와 디코더의 입력값 형식
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])

# [batch size, time steps] : 디코더의 출력값 형식
targets = tf.placeholder(tf.int64, [None, None])

# RNN 모델을 위한 셀을 구성
# : 인코더 셀과 디코더 셀로 구성
# 1.인코더 셀을 구성한다.
with tf.variable_scope('encode'):
  # 셀은 기본 셀을 사용 하였고, 각 셀에 드롭아웃을 적용
  enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
  enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
  outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                          dtype=tf.float32)
# 2.디코더 셀을 구성한다.
with tf.variable_scope('decode'):

  # 셀은 기본 셀을 사용 하였고, 각 셀에 드롭아웃을 적용
  dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
  dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
  # Seq2Seq 모델은 인코더 셀의 최종 상태값을
  # 디코더 셀의 초기 상태값으로 넣어주는 것이 핵심 아이디어
  # (텐서플로의 dynamic_rnn에 initial_state=enc_states 옵션 사용)
  outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                          initial_state=enc_states,
                                          dtype=tf.float32)

# 출력층 생성
# 출력층을 위해 layers 모듈의 dense함수를 사용
model = tf.layers.dense(outputs, n_class, activation=None)

# 비용함수
cost = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
          logits=model, labels=targets))

# 최적화 함수 : AdamOptimizer 사용
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

####################
# 신경망 모델 학습
####################
# feed_dict 로 전달하는 학습 데이터에
# 인코더의 입력값, 디코더의 입력값과 출력값 이렇게 3개를 넣어서 학습 시킨다.
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

# 100회 학습
for epoch in range(total_epoch):
  _, loss = sess.run([optimizer, cost],
                  feed_dict={enc_input: input_batch, # 인코더의 입력값
                              dec_input: output_batch, # 인코더의 출력값
                              targets: target_batch}) # 디코더의 입력값
  print('Epoch:', '%04d' %(epoch + 1),
      'cost =', '{:.6f}'.format(loss))
print('최적화 완료!')

###############
# 번역 테스트
###############
# 단어를 입력받아 번역 단어를 예측하고 디코딩하는 함수
def translate(word):
  # 이 모델은 입력값과 출력값 데이터로 [영어단어, 한글단어] 사용하지만,
  # 예측시에는 한글단어를 알지 못하므로, 디코더의 입출력값을 의미 없는 값인 P 값으로 채운다.
  # 입력으로 'word'를 받았다면, seq_data=['word', 'PPPP']로 구성될 것이다.
  seq_data = [word, 'P' * len(word)]

  # make_batch([seq_data]) 함수 호출후 다음의 결과를 리턴 받는다.
  # input_batch는 ['w','o','r','d'], output_batch는 ['P','P','P','P'] 글자들의 인덱스를 one-hot 인코딩핚 값을 리턴 받는다
  # target_batch는 ['P','P','P','P']의 각 글자의 인덱스인 [2,2,2,2]를 리턴 받는다.
  input_batch, output_batch, target_batch = make_batch([seq_data])

  # 예측 모델을 실행
  # 결과가 [batch size, time step, input] 으로 나오기 때문에,
  # 2번째 차원인 input 차원을 argmax 로 취해 가장 확률이 높은 글자를 예측 값으로 만든다.
  prediction = tf.argmax(model, 2)

  result = sess.run(prediction,
                feed_dict={enc_input: input_batch, # 인코더의 입력값
                        dec_input: output_batch, # 인코더의 출력값
                        targets: target_batch}) # 디코더의 입력값
  # 예측 결과는 글자의 인덱스를 뜻하는 숫자이므로 각 숫자에 해당하는 글자를 가져와 배열을 맊든다.
  decoded = [char_arr[i] for i in result[0]]

  # 출력의 끝을 의미하는 'E' 이후의 글자들을 제거하고 문자열로 만든다.
  end = decoded.index('E')
  translated = ''.join(decoded[:end])

  return translated

  print('\n=== 번역 테스트 ===')

# 번역함수(translate())를 사용하여 몇 단어들의 번역 결과를 리턴 받는다.
print('word ->', translate('word'))
print('wodr ->', translate('wodr'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('abcd ->', translate('abcd'))

# repaceholder 수정 필요 (70, 71)